In [ ]:
"""z_transform.ipynb"""

# Cell 01 - Generate and plot the signal

import matplotlib.pyplot as plt
import numpy as np

N_points = 1000
t = np.linspace(0, 2 * np.pi, N_points, endpoint=False)
dt = t[1] - t[0]  # time increment
y = 13 * np.exp(-t) * (17 * np.cos(3 * t) + 7 * np.sin(23 * t))

plt.figure(figsize=(8, 4))
plt.plot(t, y)
plt.xlabel("t")
plt.ylabel("y(t)")
plt.title("Sampled Data")
plt.show()

In [ ]:
# Cell 02 - Estimate the model order using SVD on a Hankel matrix


def estimate_model_order(y, L_ratio=0.5, tol_factor=1e-6):
    """
    Estimate the effective model order by constructing a Hankel matrix from y
    and computing its singular values

    Parameters:
      y         : array_like, the signal samples
      L_ratio   : float, fraction of N to use for the Hankel matrix row dimension.
                  For example, L_ratio=0.5 means L = N//2
      tol_factor: float, tolerance factor relative to the max singular value
                  to decide which singular values are significant

    Returns:
      order_est : int, estimated model order (number of significant modes)
      s         : array, singular values of the Hankel matrix
    """
    N = len(y)
    # Choose L (number of rows) as a fraction of the length. A common choice is L ~ N/2
    L = int(N * L_ratio)
    M = N - L + 1  # number of columns
    # Build the Hankel matrix H, where H[i,j] = y[i+j]
    H = np.empty((L, M))
    for i in range(L):
        H[i, :] = y[i : i + M]
    # Perform SVD on H.
    U, s, Vh = np.linalg.svd(H, full_matrices=False)
    # Determine the number of singular values above a threshold
    tol = np.max(s) * tol_factor
    order_est = np.sum(s > tol)
    return order_est, s


# Estimate the model order from the data.
estimated_order, singular_values = estimate_model_order(y, L_ratio=0.5, tol_factor=1e-6)
print("Estimated model order:", estimated_order)


In [ ]:
# Cell 03 - Given the estimated order,
# set up the linear prediction using the Z-transform approach


def z_transform_extraction(y, t, order):
    """
    Extract poles (continuous exponents) and residues using a Z-transform method.

    Parameters:
      y     : array_like, the signal samples
      t     : array_like, time samples
      order : int, model order (number of exponential terms)

    Returns:
      lam   : numpy array of continuous exponents (lambda_k = ln(z_k)/dt)
      c     : numpy array of residues for each mode
      z     : numpy array of discrete Z-poles.
    """
    N = len(y)
    dt = t[1] - t[0]
    L_pred = N - order

    # Build the linear prediction matrix: each row corresponds to
    # [y[n+order-1], y[n+order-2], ..., y[n]]
    Y_matrix = np.empty((L_pred, order))
    for n in range(L_pred):
        Y_matrix[n, :] = y[n : n + order][::-1]
    b_vector = -y[order:N]

    # Solve for the prediction coefficients using least squares
    a_est, residuals, rank, s_vals = np.linalg.lstsq(Y_matrix, b_vector, rcond=None)
    # Full polynomial: 1 + a1 z^{-1} + ... + a_order z^{-order} = 0
    coeffs = np.concatenate(([1.0], a_est))

    # Find the roots (the discrete Z-poles).
    z_poles = np.roots(coeffs)
    # Convert to continuous exponents: lambda = ln(z)/dt.
    lam = np.log(z_poles) / dt

    # Solve for the residues c_k by forming a Vandermonde matrix
    # Each mode: z_k^n for n=0,...,N-1. The Vandermonde matrix has shape (N, order)
    V = np.vander(z_poles, N, increasing=True).T  # each column corresponds to a mode
    c, residuals, rank, s_vals = np.linalg.lstsq(V, y, rcond=None)

    return lam, c, z_poles


# Use the estimated order
order = estimated_order
lam, c, z_poles = z_transform_extraction(y, t, order)

print("\nExtracted Z-poles (z):")
print(z_poles)
print("\nContinuous exponents (lambda):")
print(lam)
print("\nResidues (c):")
print(c)

In [ ]:
# Cell 04 - Group complex conjugate pairs to extract decay, frequency, amplitude, and phase

components = []
tol = 1e-2  # tolerance for matching conjugate pairs

used_indices = set()
for i in range(len(lam)):
    if i in used_indices:
        continue
    for j in range(i + 1, len(lam)):
        if np.allclose(lam[j], np.conjugate(lam[i]), atol=tol):
            used_indices.add(i)
            used_indices.add(j)
            lam_i = lam[i]
            c_i = c[i]
            decay = np.real(lam_i)  # Expected to be around -1 from the e^(-t) factor.
            frequency = np.abs(np.imag(lam_i))
            # For a conjugate pair the contribution looks like:
            # 2 |c_i| exp(decay * t) cos(frequency*t + phase)
            amplitude = 2 * np.abs(c_i)
            phase = np.angle(c_i)
            components.append(
                {
                    "decay": decay,
                    "frequency": frequency,
                    "amplitude": amplitude,
                    "phase": phase,
                }
            )
            break

print("\nExtracted Components:")
for comp in components:
    print("Component:")
    print("  Decay exponent: {:.3f}".format(comp["decay"]))
    print("  Frequency:      {:.3f} rad/s".format(comp["frequency"]))
    print("  Amplitude:      {:.3f}".format(comp["amplitude"]))
    print("  Phase:          {:.3f} rad".format(comp["phase"]))

In [ ]:
# Cell 05 - Reconstruct and plot the signal for validation

V_recon = np.vander(z_poles, N_points, increasing=True).T
y_recon = V_recon.dot(c)

plt.figure(figsize=(8, 4))
plt.plot(t, y, label="Original Signal")
plt.plot(t, np.real(y_recon), "--", label="Reconstructed Signal")
plt.xlabel("t")
plt.ylabel("y(t)")
plt.legend()
plt.title("Signal Reconstruction using Z-transform with Auto Model Order")
plt.show()